In [ ]:
# paquetes
import numpy as np
import gymnasium as gym
import mo_gymnasium as mo_gym
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
# parámetros
env = mo_gym.make("mo-lunar-lander-v3")
num_act = env.action_space.n
alpha = 0.3
gamma = 0.999
eps = 25000  # número de episodios

# exploración
epsilon_i = 1.0  # inicial
epsilon_f = 0.01  # final
epsilon_d = 0.9999  # decay

In [ ]:
# crea Q_set que guarda para cada estado 
_ = env.reset()
_, r_vec, _, _, _ = env.step(env.action_space.sample())
num_obj = len(r_vec)
Q = defaultdict(lambda: np.zeros((num_act, num_obj)))  

# diccs
front = []                   # frente de Pareto
r_hist = []                  # Recompensa por episodio
pareto_t_hist = []           # tamaño del frente de Pareto
t_traj = {}     # dicc de trayectorias que alcanzan el frente de Pareto

In [ ]:
def discret(estado):
    return tuple(np.round(estado, decimals=1))

def selc_accion(estado, epsilon=0.1):
    q_valores = Q[estado]
    acc_nodom = [] # acciones no dominadas
    for a in range(num_act):
        dom = False
        for b in range(num_act):
            if a != b:
                if np.all(q_valores[b] >= q_valores[a]) and np.any(q_valores[b] > q_valores[a]):
                    dom = True
                    break
        if not dom:
            acc_nodom.append(a)
    if np.random.rand() < epsilon or len(acc_nodom) == 0:
        return np.random.randint(num_act)
    else:
        return np.random.choice(acc_nodom)

In [ ]:
# actualizar frente de Pareto
def act(frontera, nuevo_vec):
    no_dominados = []
    for vec in frontera:
        if np.all(vec >= nuevo_vec) and np.any(vec > nuevo_vec):
            return frontera  
        elif not (np.all(nuevo_vec >= vec) and np.any(nuevo_vec > vec)):
            no_dominados.append(vec)
    no_dominados.append(nuevo_vec)
    return no_dominados

In [ ]:

for ep in range(eps):
    epsilon_a = max(epsilon_f, epsilon_i * (epsilon_d ** ep))
    
    estado, _ = env.reset()
    estado = discret(estado)
    terminado = False
    r_acc = np.zeros(num_obj)  
    trajectory = []  
    
    while not terminado:
        accion = selc_accion(estado, epsilon=epsilon_a)
        nv_estado, r_vec, fin, tronco, _ = env.step(accion)
        nv_estado = discret(nv_estado)
        terminado = fin or tronco
        
        mejor_sig = np.max(Q[nv_estado], axis=0)
        Q[estado][accion] = (1 - alpha) * Q[estado][accion] + alpha * (np.array(r_vec) + gamma * mejor_sig)
        
        trajectory.append((estado, accion, np.array(r_vec), nv_estado))
        
        estado = nv_estado
        r_acc += np.array(r_vec)
    
    r_hist.append(r_acc)
    front = act(front, r_acc)
    pareto_t_hist.append(len(front))
    
    if np.isclose(r_acc[0], 100, atol=1e-3) and r_acc[0] > 0:
        t_traj[ep] = trajectory
        
    if ep % 100 == 0:
        print(f"Episodio {ep} (ε = {epsilon_a:.3f}), recompensa: {r_acc}")

env.close()